# 通过示例学习PyTorch

本教程通过独立的示例介绍 ```<u>PyTorch<\u>``` 的基本概念。

PyTorch 的核心是提供两个主要功能：
> * n 维张量，类似于 NumPy，但可以在 GPU 上运行
> * 用于构建和训练神经网络的自动微分

我们将使用将三阶多项式拟合 ```y = sin(x)``` 的问题作为运行示例。 该网络将具有四个参数，并且将通过使网络输出与实际输出之间的欧几里德距离最小化来进行梯度下降训练，以适应随机数据。

注意

您可以在```本页```浏览各个示例。

## Ⅰ.预热：NumPy

在介绍 PyTorch 之前，我们将首先使用 numpy 实现网络。

Numpy 提供了一个 n 维数组对象，以及许多用于操纵这些数组的函数。 Numpy 是用于科学计算的通用框架。 它对计算图，深度学习或梯度一无所知。 但是，通过使用 numpy 操作手动实现网络的前向和后向传递，我们可以轻松地使用 numpy 使三阶多项式适合正弦函数：

In [3]:
# -*- coding: utf-8 -*-
import numpy as np
import math

# Create random input and output data
x = np.linspace(-math.pi, math.pi, 2000)
y = np.sin(x)

# Randomly initialize weights
# 随机初始化权重
a = np.random.randn()
b = np.random.randn()
c = np.random.randn()
d = np.random.randn()

learning_rate = 1e-6
for t in range(2000):
    # Forward pass: compute predicted y
    # 正向传递：计算预测 y
    # y = a + b x + c x^2 + d x^3
    y_pred = a + b * x + c * x ** 2 + d * x ** 3

    # Compute and print loss
    loss = np.square(y_pred - y).sum()
    if t % 100 == 99:
        print(t, loss)

    # Backprop to compute gradients of a, b, c, d with respect to loss
    grad_y_pred = 2.0 * (y_pred - y)
    grad_a = grad_y_pred.sum()
    grad_b = (grad_y_pred * x).sum()
    grad_c = (grad_y_pred * x ** 2).sum()
    grad_d = (grad_y_pred * x ** 3).sum()

    # Update weights
    a -= learning_rate * grad_a
    b -= learning_rate * grad_b
    c -= learning_rate * grad_c
    d -= learning_rate * grad_d

print(f'Result: y = {a} + {b} x + {c} x^2 + {d} x^3')


99 3267.156712057645
199 2232.6351478785537
299 1528.3182050261917
399 1048.2632717706554
499 720.6895515642644
599 496.9076593771385
699 343.8553352089778
799 239.05704537173096
899 167.21697048762707
999 117.91371822292233
1099 84.03887062103483
1199 60.738232691594746
1299 44.69316361335828
1399 33.63221626564388
1499 25.9989097818173
1599 20.725463614579134
1699 17.07852244400094
1799 14.553846410028086
1899 12.80434305153441
1999 11.590828192638812
Result: y = -0.04840604083464015 + 0.83134880554618 x + 0.008350847523971711 x^2 + -0.0897185928181053 x^3


## Ⅱ.PyTorch：张量

Numpy 是一个很棒的框架，但是它不能利用 GPU 来加速其数值计算。 对于现代深度神经网络，GPU 通常会提供 ```50 倍或更高``` 的加速，因此遗憾的是，numpy 不足以实现现代深度学习。

在这里，我们介绍最基本的 PyTorch 概念：**张量**。 PyTorch 张量在概念上与 numpy 数组相同：张量是 n 维数组，PyTorch 提供了许多在这些张量上进行操作的函数。 在幕后，张量可以跟踪计算图和梯度，但它们也可用作科学计算的通用工具。

与 numpy 不同，PyTorch 张量可以利用 GPU 加速其数字计算。 要在 GPU 上运行 PyTorch 张量，您只需要指定正确的设备即可。

在这里，我们使用 PyTorch 张量将三阶多项式拟合为正弦函数。 像上面的 numpy 示例一样，我们需要手动实现通过网络的正向和反向传递：

In [13]:
import torch
import math

dtype = torch.float
device = torch.device("cpu")
# device = torch.device("cuda:0") # Uncomment this to run on GPU()  (取消注释以在 GPU 上运行)

# Create random input and output data
x = torch.linspace(-math.pi, math.pi, 2000, device=device, dtype=dtype)
y = torch.sin(x)

# Randomly initialize weights
# 随机初始化权重
a = torch.randn((), device=device, dtype=dtype)
b = torch.randn((), device=device, dtype=dtype)
c = torch.randn((), device=device, dtype=dtype)
d = torch.randn((), device=device, dtype=dtype)

learning_rate = 1e-6
for t in range(2000):
    # Forward pass: compute predicted y
    # 正向传递：计算预测 y
    y_pred = a + b * x + c * x ** 2 + d * x ** 3

    # Compute and print loss
    loss = (y_pred - y).pow(2).sum().item()
    if t % 100 == 99:
        print(t, loss)

    # Backprop to compute gradients of a, b, c, d with respect to loss
    # 反向螺旋体，用于计算 a、b、c、d 相对于损耗的梯度
    grad_y_pred = 2.0 * (y_pred - y)
    grad_a = grad_y_pred.sum()
    grad_b = (grad_y_pred * x).sum()
    grad_c = (grad_y_pred * x ** 2).sum()
    grad_d = (grad_y_pred * x ** 3).sum()

    # Update weights using gradient descent
    # 使用梯度下降更新权重
    a -= learning_rate * grad_a
    b -= learning_rate * grad_b
    c -= learning_rate * grad_c
    d -= learning_rate * grad_d

print(f'Result: y = {a.item()} + {b.item()} x + {c.item()} x^2 + {d.item()} x^3')


99 1455.956787109375
199 991.6073608398438
299 676.9944458007812
399 463.6046447753906
499 318.713134765625
599 220.22357177734375
699 153.20050048828125
799 107.5394515991211
899 76.39625549316406
999 55.13068389892578
1099 40.593284606933594
1199 30.643871307373047
1299 23.826614379882812
1399 19.15015411376953
1499 15.938590049743652
1599 13.73052978515625
1699 12.210708618164062
1799 11.16346263885498
1899 10.44105339050293
1999 9.9421968460083
Result: y = 0.02958722785115242 + 0.8387191891670227 x + -0.005104289390146732 x^2 + -0.09076696634292603 x^3


## Ⅲ.PyTorch：张量 与 Autograd

这里我们准备一个三阶多项式，通过最小化平方欧几里得距离来训练，并预测函数 ```y = sin(x)``` 在 ```-pi``` 到 ```pi``` 上的值。

此实现使用了 PyTorch 张量(tensor)运算来实现前向传播，并使用 PyTorch Autograd 来计算梯度。

PyTorch 张量表示计算图中的一个节点。 如果x是一个张量，且 ```x.requires_grad=True``` ，则 ```x.grad``` 是另一个张量，它保存了x相对于某个标量值的梯度。

In [6]:
import torch
import math

dtype = torch.float
# device = torch.device("cpu")
device = torch.device("cuda:0")  # Uncomment this to run on GPU

# Create Tensors to hold input and outputs.
# By default, requires_grad=False, which indicates that we do not need to
# compute gradients with respect to these Tensors during the backward pass.
# 创建张量来保存输入和输出。
# 默认情况下，requires_grad=False，这表明在向后传递期间，我们不需要计算相对于这些张量的梯度。
x = torch.linspace(-math.pi, math.pi, 2000, device=device, dtype=dtype)
y = torch.sin(x)

# Create random Tensors for weights. For a third order polynomial, we need
# 4 weights: y = a + b x + c x^2 + d x^3
# Setting requires_grad=True indicates that we want to compute gradients with
# respect to these Tensors during the backward pass.
# 为权重创建随机张量。
# 对于三阶多项式，我们需要 4 个权重：y = a + b x + c x^2 + d x^3 
# 设置 requires_grad=True 表示我们要在向后传递期间计算相对于这些张量的梯度。
a = torch.randn((), device=device, dtype=dtype, requires_grad=True)
b = torch.randn((), device=device, dtype=dtype, requires_grad=True)
c = torch.randn((), device=device, dtype=dtype, requires_grad=True)
d = torch.randn((), device=device, dtype=dtype, requires_grad=True)

learning_rate = 1e-6
for t in range(2000):
    # Forward pass: compute predicted y using operations on Tensors.
    # 正向传递：使用张量上的运算计算预测的 y。
    y_pred = a + b * x + c * x ** 2 + d * x ** 3

    # Compute and print loss using operations on Tensors.
    # Now loss is a Tensor of shape (1,)
    # loss.item() gets the scalar value held in the loss.
    # loss.item（） 获取损失中持有的标量值。
    loss = (y_pred - y).pow(2).sum()
    if t % 100 == 99:
        print(t, loss.item())

    # Use autograd to compute the backward pass. This call will compute the
    # gradient of loss with respect to all Tensors with requires_grad=True.
    # After this call a.grad, b.grad. c.grad and d.grad will be Tensors holding
    # the gradient of the loss with respect to a, b, c, d respectively.
    # 使用自动渐变来计算向后传递。
    # 此调用将计算相对于所有张量的损失梯度，requires_grad=True。
    # 在此调用a.grad，b.grad之后。
    # c.grad 和 d.grad 将分别是保持相对于 a、b、c、d 的损失梯度的张量。
    loss.backward()

    # Manually update weights using gradient descent. Wrap in torch.no_grad()
    # because weights have requires_grad=True, but we don't need to track this
    # in autograd.
    # 使用梯度下降手动更新权重。
    # 换行torch.no_grad（），因为权重requires_grad=True，但我们不需要在自动升级中跟踪它。
    with torch.no_grad():
        a -= learning_rate * a.grad
        b -= learning_rate * b.grad
        c -= learning_rate * c.grad
        d -= learning_rate * d.grad

        # Manually zero the gradients after updating weights
        # 更新权重后手动清零渐变
        a.grad = None
        b.grad = None
        c.grad = None
        d.grad = None

print(f'Result: y = {a.item()} + {b.item()} x + {c.item()} x^2 + {d.item()} x^3')


99 1503.9141845703125
199 1011.8036499023438
299 682.1997680664062
399 461.2796325683594
499 313.0946044921875
599 213.62030029296875
699 146.7905731201172
799 101.8546142578125
899 71.61383056640625
999 51.24409866333008
1099 37.51081848144531
1199 28.242835998535156
1299 21.982315063476562
1399 17.74897003173828
1499 14.88352108001709
1599 12.941917419433594
1699 11.624869346618652
1799 10.730536460876465
1899 10.122562408447266
1999 9.708781242370605
Result: y = -0.022038761526346207 + 0.8359349966049194 x + 0.0038020508363842964 x^2 + -0.09037093073129654 x^3


## Ⅳ.PyTorch：定义新的 Autograd 函数

这里我们准备一个三阶多项式，通过最小化平方欧几里得距离来训练，并预测函数<font color="orange"> $y = sin(x)$ </font>在<font color="orange"> $-pi$ </font>到<font color="orange"> $pi$ </font>上的值。

这里我们不将多项式写为<font color="orange"> $y = a + bx + cx^2 + dx^3$ </font>，而是将多项式写为<font color="orange"> $y = a + bP_3(c + dx)$ </font>，其中<font color="orange"> $P_3(x) = 1/2 (5x ^ 3 - 3x)$ </font>是```三次勒让德多项式。```

此实现使用了 PyTorch 张量(tensor)运算来实现前向传播，并使用 PyTorch Autograd 来计算梯度。

在此实现中，我们实现了自己的自定义 Autograd 函数来执行<font color="orange"> $P'_3(x)$ </font>。 从数学定义上讲，<font color="orange">$ P'_3(x) = 3/2 (5x ^ 2 - 1)$</font> ：

In [7]:
import torch
import math

class LegendrePolynomial3(torch.autograd.Function):
    """
    We can implement our own custom autograd Functions by subclassing
    torch.autograd.Function and implementing the forward and backward passes
    which operate on Tensors.

    我们可以通过子类化 torch.autograd.Function 
    并实现在 Tensors 上运行的向前和向后传递
    来实现我们自己的自定义 autograd 函数。
    """

    # python staticmethod 返回函数的静态方法
    @staticmethod
    def forward(ctx, input):
        """
        In the forward pass we receive a Tensor containing the input and return
        a Tensor containing the output. ctx is a context object that can be used
        to stash information for backward computation. You can cache arbitrary
        objects for use in the backward pass using the ctx.save_for_backward method.

        在正向传递中，我们接收到包含输入的张量，并返回包含输出的张量。
        ctx 是一个上下文对象，可用于隐藏信息以进行向后计算。
        您可以使用ctx.save_for_backward方法缓存任意对象，以便在向后传递中使用。
        """
        ctx.save_for_backward(input)
        return 0.5 * (5 * input ** 3 - 3 * input)

    @staticmethod
    def backward(ctx, grad_output):
        """
        In the backward pass we receive a Tensor containing the gradient of the loss
        with respect to the output, and we need to compute the gradient of the loss
        with respect to the input.

        在反向传递中，我们收到一个包含相对于输出的损耗梯度的张量，并且我们需要计算相对于输入的损耗梯度。
        """
        input, = ctx.saved_tensors
        return grad_output * 1.5 * (5 * input ** 2 - 1)

dtype = torch.float
# device = torch.device("cpu")
device = torch.device("cuda:0")  # Uncomment this to run on GPU

# Create Tensors to hold input and outputs.
# By default, requires_grad=False, which indicates that we do not need to
# compute gradients with respect to these Tensors during the backward pass.
# 创建张量来保存输入和输出。
# 默认情况下，requires_grad=False，
# 这表明在向后传递期间，我们不需要计算相对于这些张量的梯度。
x = torch.linspace(-math.pi, math.pi, 2000, device=device, dtype=dtype)
y = torch.sin(x)

# Create random Tensors for weights. For this example, we need
# 4 weights: y = a + b * P3(c + d * x), these weights need to be initialized
# not too far from the correct result to ensure convergence.
# Setting requires_grad=True indicates that we want to compute gradients with
# respect to these Tensors during the backward pass.
# 为权重创建随机张量。
# 对于此示例，我们需要 4 个权重：y = a + b * P3（c + d * x），这些权重需要在距离正确结果不太远的地方进行初始化，以确保收敛。
# 设置 requires_grad=True 表示我们希望在向后传递期间计算相对于这些张量的梯度。
a = torch.full((), 0.0, device=device, dtype=dtype, requires_grad=True)
b = torch.full((), -1.0, device=device, dtype=dtype, requires_grad=True)
c = torch.full((), 0.0, device=device, dtype=dtype, requires_grad=True)
d = torch.full((), 0.3, device=device, dtype=dtype, requires_grad=True)

learning_rate = 5e-6
for t in range(2000):
    # To apply our Function, we use Function.apply method. We alias this as 'P3'.
    # 为了应用我们的函数，我们使用Function.apply方法。我们将其别名为"P3"。
    P3 = LegendrePolynomial3.apply

    # Forward pass: compute predicted y using operations; we compute
    # P3 using our custom autograd operation.
    # 正向传递：使用运算计算预测的y;
    #         我们使用自定义自动升级操作计算 P3。
    y_pred = a + b * P3(c + d * x)

    # Compute and print loss
    loss = (y_pred - y).pow(2).sum()
    if t % 100 == 99:
        print(t, loss.item())

    # Use autograd to compute the backward pass.
    loss.backward()

    # Update weights using gradient descent
    with torch.no_grad():
        a -= learning_rate * a.grad
        b -= learning_rate * b.grad
        c -= learning_rate * c.grad
        d -= learning_rate * d.grad

        # Manually zero the gradients after updating weights
        a.grad = None
        b.grad = None
        c.grad = None
        d.grad = None

print(f'Result: y = {a.item()} + {b.item()} * P3({c.item()} + {d.item()} x)')


99 209.95834350585938
199 144.66018676757812
299 100.70249938964844
399 71.03520202636719
499 50.978515625
599 37.40313720703125
699 28.20686912536621
799 21.973186492919922
899 17.745729446411133
999 14.877889633178711
1099 12.931766510009766
1199 11.610918998718262
1299 10.714248657226562
1399 10.105474472045898
1499 9.692106246948242
1599 9.411375045776367
1699 9.220745086669922
1799 9.091285705566406
1899 9.003361701965332
1999 8.943639755249023
Result: y = -1.765793067320942e-11 + -2.208526849746704 * P3(9.924167737596079e-11 + 0.2554861009120941 x)


## Ⅴ.PyTorch: nn

一个三阶多项式，通过最小化平方欧几里得距离来训练，并预测函数<font color = "orange"> $y = sin(x)$ </font>在<font color = "orange"> $-pi$ </font>到<font color = "orange"> $pi$ </font>上的值。

这个实现使用 PyTorch 的 <font color = "orange">nn</font> 包来构建神经网络。 PyTorch Autograd 让我们定义计算图和计算梯度变得容易了，但是原始的 Autograd 对于定义复杂的神经网络来说可能太底层了。 这时候 <font color = "orange">nn</font> 包就能帮上忙。 <font color = "orange">nn</font> 包定义了一组模块，你可以把它视作一层神经网络，该神经网络层接受输入，产生输出，并且可能有一些可训练的权重。

In [17]:
import torch
import math

# Create Tensors to hold input and outputs.
x = torch.linspace(-math.pi, math.pi, 2000)
y = torch.sin(x)

# For this example, the output y is a linear function of (x, x^2, x^3), so
# we can consider it as a linear layer neural network. Let's prepare the
# tensor (x, x^2, x^3).
# 对于此示例，输出 y 是 （x， x^2， x^3） 的线性函数，
# 因此我们可以将其视为线性层神经网络。
# 让我们准备张量（x， x^2， x^3）。
p = torch.tensor([1, 2, 3])
xx = x.unsqueeze(-1).pow(p)

# In the above code, x.unsqueeze(-1) has shape (2000, 1), and p has shape
# (3,), for this case, broadcasting semantics will apply to obtain a tensor
# of shape (2000, 3)
# 在上面的代码中，
# x.unsqueeze（-1） 有 shape （2000， 1），
# p 有 shape （3，），
# 对于这种情况，广播语义将应用于获得形状的张量 （2000， 3）

# Use the nn package to define our model as a sequence of layers. nn.Sequential
# is a Module which contains other Modules, and applies them in sequence to
# produce its output. The Linear Module computes output from input using a
# linear function, and holds internal Tensors for its weight and bias.
# The Flatten layer flatens the output of the linear layer to a 1D tensor,
# to match the shape of `y`.
# 使用 nn 包将我们的模型定义为一系列层。
# 嗯。顺序是一个模块，其中包含其他模块，并按顺序应用它们以生成其输出。
# 线性模块使用线性函数计算输入输出，并保存其权重和偏差的内部张量。
# 展平层将线性层的输出平展为 1D 张量，以匹配 "y" 的形状。
model = torch.nn.Sequential(
    torch.nn.Linear(3, 1),
    torch.nn.Flatten(0, 1)
)

# The nn package also contains definitions of popular loss functions; in this
# case we will use Mean Squared Error (MSE) as our loss function.
# nn 包还包含常用损失函数的定义;在这种情况下，我们将使用均方误差（MSE）作为我们的损失函数。
loss_fn = torch.nn.MSELoss(reduction='sum')

learning_rate = 1e-6
for t in range(2000):

    # Forward pass: compute predicted y by passing x to the model. Module objects
    # override the __call__ operator so you can call them like functions. When
    # doing so you pass a Tensor of input data to the Module and it produces
    # a Tensor of output data.
    # 正向传递：通过将 x 传递给模型来计算预测的 y。
    #         模块对象将覆盖__call__运算符，以便您可以像调用函数一样调用它们。
    #         这样做时，您将输入数据的张量传递给模块，它会生成输出数据的张量。
    y_pred = model(xx)

    # Compute and print loss. We pass Tensors containing the predicted and true
    # values of y, and the loss function returns a Tensor containing the
    # loss.
    # 计算和打印丢失。
    # 我们传递包含 y 的预测值和真实值的张量，损失函数返回包含损失的张量。
    loss = loss_fn(y_pred, y)
    if t % 100 == 99:
        print(t, loss.item())

    # Zero the gradients before running the backward pass.
    # 在运行向后通道之前将渐变清零。
    model.zero_grad()

    # Backward pass: compute gradient of the loss with respect to all the learnable
    # parameters of the model. Internally, the parameters of each Module are stored
    # in Tensors with requires_grad=True, so this call will compute gradients for
    # all learnable parameters in the model.
    # 向后传递：计算相对于模型所有可学习参数的损失梯度。
    # 在内部，每个模块的参数都存储在张量中，requires_grad=True，因此此调用将计算模型中所有可学习参数的梯度。
    loss.backward()

    # Update the weights using gradient descent. Each parameter is a Tensor, so
    # we can access its gradients like we did before.
    # 使用梯度下降更新权重。
    # 每个参数都是一个张量，因此我们可以像以前一样访问其梯度
    with torch.no_grad():
        for param in model.parameters():
            param -= learning_rate * param.grad

# You can access the first layer of `model` like accessing the first item of a list
# 您可以访问"模型"的第一层，就像访问列表的第一项一样
linear_layer = model[0]

# For linear layer, its parameters are stored as `weight` and `bias`.
# 对于线性层，其参数存储为"权重"和"偏差"。
print(f'Result: y = {linear_layer.bias.item()} + {linear_layer.weight[:, 0].item()} x + {linear_layer.weight[:, 1].item()} x^2 + {linear_layer.weight[:, 2].item()} x^3')


99 288.37646484375
199 198.5834503173828
299 137.7730712890625
399 96.54682159423828
499 68.56768798828125
599 49.55795669555664
699 36.627952575683594
799 27.823333740234375
899 21.821151733398438
999 17.724679946899414
1099 14.925694465637207
1199 13.011001586914062
1299 11.699724197387695
1399 10.800652503967285
1499 10.18351936340332
1599 9.759430885314941
1699 9.467667579650879
1799 9.266709327697754
1899 9.128161430358887
1999 9.032530784606934
Result: y = -0.012886816635727882 + 0.8487757444381714 x + 0.0022231899201869965 x^2 + -0.09219741821289062 x^3


## Ⅵ.PyTorch: optim

经过训练的三阶多项式，可以通过最小化平方的欧几里得距离来预测 <font color="orange">$y = sin(x)$</font>从 <font color="orange">$-pi$</font> 到 <font color="orange">$pi$</font>。

此实现使用来自 PyTorch 的 <font color="orange">nn</font> 包来构建网络。

与其像以前那样手动更新模型的权重，不如使用 <font color="orange">optim</font> 包定义一个优化器，该优化器将为我们更新权重。 <font color="orange">optim</font> 包定义了许多深度学习常用的优化算法，包括 SGD + 动量，RMSProp，Adam 等。

In [20]:
import torch
import math

# Create Tensors to hold input and outputs.
x = torch.linspace(-math.pi, math.pi, 2000)
y = torch.sin(x)

# Prepare the input tensor (x, x^2, x^3).
p = torch.tensor([1, 2, 3])
xx = x.unsqueeze(-1).pow(p)

# Use the nn package to define our model and loss function.
model = torch.nn.Sequential(
    torch.nn.Linear(3, 1),
    torch.nn.Flatten(0, 1)
)
loss_fn = torch.nn.MSELoss(reduction='sum')

# Use the optim package to define an Optimizer that will update the weights of
# the model for us. Here we will use RMSprop; the optim package contains many other
# optimization algorithms. The first argument to the RMSprop constructor tells the
# optimizer which Tensors it should update.
learning_rate = 1e-3
optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate)
for t in range(2000):
    # Forward pass: compute predicted y by passing x to the model.
    y_pred = model(xx)

    # Compute and print loss.
    loss = loss_fn(y_pred, y)
    if t % 100 == 99:
        print(t, loss.item())

    # Before the backward pass, use the optimizer object to zero all of the
    # gradients for the variables it will update (which are the learnable
    # weights of the model). This is because by default, gradients are
    # accumulated in buffers( i.e, not overwritten) whenever .backward()
    # is called. Checkout docs of torch.autograd.backward for more details.
    optimizer.zero_grad()

    # Backward pass: compute gradient of the loss with respect to model
    # parameters
    loss.backward()

    # Calling the step function on an Optimizer makes an update to its
    # parameters
    optimizer.step()

linear_layer = model[0]
print(f'Result: y = {linear_layer.bias.item()} + {linear_layer.weight[:, 0].item()} x + {linear_layer.weight[:, 1].item()} x^2 + {linear_layer.weight[:, 2].item()} x^3')

99 6489.7978515625
199 1493.303955078125
299 313.51251220703125
399 123.85812377929688
499 70.52151489257812
599 38.60538864135742
699 20.43478775024414
799 11.971320152282715
899 9.273175239562988
999 8.841493606567383
1099 8.817790985107422
1199 8.817176818847656
1299 8.817168235778809
1399 9.030593872070312
1499 8.940629959106445
1599 9.032800674438477
1699 9.050511360168457
1799 8.937700271606445
1899 8.88145637512207
1999 8.905050277709961
Result: y = -0.00046780623961240053 + 0.8571969866752625 x + -0.00046821858268231153 x^2 + -0.09287428855895996 x^3


## Ⅶ.PyTorch:自定义nn模块

经过训练的三阶多项式，可以通过最小化平方的欧几里得距离来预测 <font color=orange>$y = sin(x)$</font> 从 <font color=orange>$-pi$</font> 到 <font color=orange>$pi$</font> 。

此实现将模型定义为自定义 <font color=orange>$Module$</font> 子类。 每当您想要一个比现有模块的简单序列更复杂的模型时，都需要以这种方式定义模型。

In [21]:
import torch
import math

class Polynomial3(torch.nn.Module):
    def __init__(self):
        """
        In the constructor we instantiate four parameters and assign them as
        member parameters.

        在构造函数中，我们实例化四个参数，并将它们作为成员参数赋值。
        """
        super().__init__()
        self.a = torch.nn.Parameter(torch.randn(()))
        self.b = torch.nn.Parameter(torch.randn(()))
        self.c = torch.nn.Parameter(torch.randn(()))
        self.d = torch.nn.Parameter(torch.randn(()))

    def forward(self, x):
        """
        In the forward function we accept a Tensor of input data and we must return
        a Tensor of output data. We can use Modules defined in the constructor as
        well as arbitrary operators on Tensors.

        在正向函数中，我们接受输入数据的张量，并且我们必须返回输出数据的张量。
        我们可以使用构造函数中定义的模块以及张量上的任意运算符。
        """
        return self.a + self.b * x + self.c * x ** 2 + self.d * x ** 3

    def string(self):
        """
        Just like any class in Python, you can also define custom method on PyTorch modules

        就像Python中的任何类一样，你也可以在PyTorch模块上定义自定义方法。
        """
        return f'y = {self.a.item()} + {self.b.item()} x + {self.c.item()} x^2 + {self.d.item()} x^3'

# Create Tensors to hold input and outputs.
x = torch.linspace(-math.pi, math.pi, 2000)
y = torch.sin(x)

# Construct our model by instantiating the class defined above
# 通过实例化上面定义的类来构造我们的模型
model = Polynomial3()

# Construct our loss function and an Optimizer. The call to model.parameters()
# in the SGD constructor will contain the learnable parameters of the nn.Linear
# module which is members of the model.
# 构建我们的损失函数和优化器。
# SGD 构造函数中对 model.parameters（） 的调用将包含 nn 的可学习参数。
# 作为模型成员的线性模块。
criterion = torch.nn.MSELoss(reduction='sum')
optimizer = torch.optim.SGD(model.parameters(), lr=1e-6)
for t in range(2000):
    # Forward pass: Compute predicted y by passing x to the model
    y_pred = model(x)

    # Compute and print loss
    loss = criterion(y_pred, y)
    if t % 100 == 99:
        print(t, loss.item())

    # Zero gradients, perform a backward pass, and update the weights.
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

print(f'Result: {model.string()}')


99 108.68385314941406
199 74.85714721679688
299 52.488563537597656
399 37.696712493896484
499 27.91497802734375
599 21.446483612060547
699 17.16891860961914
799 14.340225219726562
899 12.469612121582031
999 11.232577323913574
1099 10.414496421813965
1199 9.873517036437988
1299 9.515730857849121
1399 9.279145240783691
1499 9.12269115447998
1599 9.019227027893066
1699 8.950798034667969
1799 8.905545234680176
1899 8.875618934631348
1999 8.855826377868652
Result: y = 0.0003440560249146074 + 0.8627766966819763 x + -5.9356367273721844e-05 x^2 + -0.09418893605470657 x^3


## Ⅷ.控制流+权重共享

为了展示 PyTorch 动态图的强大功能，我们将实现一个非常奇怪的模型：一个从三到五阶（动态变化）的多项式，在每次正向传递中选择一个 3 到 5 之间的一个随机数，并将这个随机数作为阶数，第四和第五阶共用同一个权重来多次重复计算。

In [23]:
import random
import torch
import math

class DynamicNet(torch.nn.Module):
    def __init__(self):
        """
        In the constructor we instantiate five parameters and assign them as members.

        在构造函数中，我们实例化五个参数，并将它们作为成员进行分配。
        """
        super().__init__()
        self.a = torch.nn.Parameter(torch.randn(()))
        self.b = torch.nn.Parameter(torch.randn(()))
        self.c = torch.nn.Parameter(torch.randn(()))
        self.d = torch.nn.Parameter(torch.randn(()))
        self.e = torch.nn.Parameter(torch.randn(()))

    def forward(self, x):
        """
        For the forward pass of the model, we randomly choose either 4, 5
        and reuse the e parameter to compute the contribution of these orders.
        对于模型的正向传递，我们随机选择 4、5，然后重用 e 参数来计算这些订单的贡献。

        Since each forward pass builds a dynamic computation graph, we can use normal
        Python control-flow operators like loops or conditional statements when
        defining the forward pass of the model.
        由于每个正向传递都构建了一个动态计算图，因此在定义模型的前向传递时，我们可以使用普通的 Python 控制流运算符，如循环或条件语句。

        Here we also see that it is perfectly safe to reuse the same parameter many
        times when defining a computational graph.
        在这里，我们还看到，在定义计算图时，多次重用相同的参数是完全安全的。
        """
        y = self.a + self.b * x + self.c * x ** 2 + self.d * x ** 3
        for exp in range(4, random.randint(4, 6)):
            y = y + self.e * x ** exp
        return y

    def string(self):
        """
        Just like any class in Python, you can also define custom method on PyTorch modules

        就像Python中的任何类一样，你也可以在PyTorch模块上定义自定义方法。
        """
        return f'y = {self.a.item()} + {self.b.item()} x + {self.c.item()} x^2 + {self.d.item()} x^3 + {self.e.item()} x^4 ? + {self.e.item()} x^5 ?'

# Create Tensors to hold input and outputs.
x = torch.linspace(-math.pi, math.pi, 2000)
y = torch.sin(x)

# Construct our model by instantiating the class defined above
# 通过实例化上面定义的类来构造我们的模型
model = DynamicNet()

# Construct our loss function and an Optimizer. Training this strange model with
# vanilla stochastic gradient descent is tough, so we use momentum
# 构建我们的损失函数和优化器。用香草随机梯度下降训练这个奇怪的模型是很困难的，所以我们使用动量
criterion = torch.nn.MSELoss(reduction='sum')
optimizer = torch.optim.SGD(model.parameters(), lr=1e-8, momentum=0.9)
for t in range(30000):
    # Forward pass: Compute predicted y by passing x to the model
    y_pred = model(x)

    # Compute and print loss
    loss = criterion(y_pred, y)
    if t % 2000 == 1999:
        print(t, loss.item())

    # Zero gradients, perform a backward pass, and update the weights.
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

print(f'Result: {model.string()}')


1999 1795.8043212890625
3999 817.8870239257812
5999 476.62420654296875
7999 196.5718231201172
9999 99.62507629394531
11999 51.38410949707031
13999 29.43459701538086
15999 18.814685821533203
17999 13.621737480163574
19999 11.303443908691406
21999 10.935626983642578
23999 9.355867385864258
25999 8.912134170532227
27999 8.956872940063477
29999 8.663052558898926
Result: y = -0.007251318544149399 + 0.8541530966758728 x + 0.0008736016461625695 x^2 + -0.09326013177633286 x^3 + 9.603642683941871e-05 x^4 ? + 9.603642683941871e-05 x^5 ?
